In [1]:
import datasets
import tqdm

import numpy
numpy.object = object

## Load dataset

These are [Huggingface dataset](https://huggingface.co/docs/datasets/en/index) formats.

In [2]:
opt_dataset = datasets.Dataset.load_from_disk("data/additional-optimizations/")
td_dataset = datasets.Dataset.load_from_disk("data/additional-torsiondrives/")

Datasets are represented with the features in each entry.

Datasets can be indexed to get a single entry. "coords", "forces", etc. are stored as flat lists of floats.

We'll need to re-convert to PyTorch.

In [3]:
# reformat dataset lists to torch tensors
opt_dataset.set_format(
    "torch", columns=["energy", "coords", "forces"], output_all_columns=True
)
td_dataset.set_format(
    "torch", columns=["energy", "coords", "forces"], output_all_columns=True
)

In [4]:
len(opt_dataset)

70

## Hessian dataset from json

In [5]:
import json


def unwrap(iterable):
    i = iter(iterable)
    ret = next(i)
    try:
        next(i)
    except StopIteration:
        return ret
    raise ValueError("Multiple elements")


with open("data/additional-hessians.json", "r") as f:
    contents = json.load(f)

hess_dataset = [
    {"smiles": smiles, **{k: [conf[k] for conf in confs] for k in confs[0]}}
    for smiles, confs in contents.items()
]

In [6]:
for entry in opt_dataset:
    hess = contents[entry["smiles"]]
    assert len(entry["energy"]) == len(hess)
    assert (
        len(hess[0]["hessian"]) == (len(entry["coords"]) // len(entry["energy"])) ** 2
    )

## Fitting

For how to fit a force field to optimization data from a SMIRNOFF force field, here's an [example I put together for the IRL Irvine meeting](https://openforcefield.atlassian.net/wiki/spaces/MEET/pages/3440508935/Hackathon+How+to+train+your+force+field+with+smee) (`run-smee-fit-from-qca-data-commented.ipynb` where you can largely follow on from the "Assign parameters to molecules in the dataset" heading.

The only note is that the `descent.targets.energy.predict` function would have to be re-written to not include forces in the objective and prediction if they're not in the data.

In [7]:
from utils import ff_to_csys

chemical_system = ff_to_csys("openff-2.2.1.offxml")

In [8]:
import numpy as np
from besmarts.core.assignments import graph_db as GraphDb
from besmarts.core.assignments import graph_db_add_single_molecule_state
from besmarts.core.assignments import graph_db_address as GraphDbAddress
from besmarts.mechanics.fits import (
    objective_config_energy_total as ObjectiveConfigEnergyTotal,
)
from besmarts.mechanics.fits import objective_config_gradient as ObjectiveConfigGradient
from besmarts.mechanics.fits import objective_config_hessian as ObjectiveConfigHessian
from besmarts.mechanics.fits import objective_config_position as ObjectiveConfigPosition
from besmarts.mechanics.fits import objective_tier as ObjectiveTier
from openff.toolkit import Molecule
from utils import data_to_graph_assignment

graph_db = GraphDb()
eid = 0
objectives = []
smiles_to_opt_eids = {}
smiles_to_add = set(opt_dataset["smiles"][:10])

# First, the optimizations
for entry in hess_dataset:
    smiles = entry["smiles"]
    if smiles not in smiles_to_add:
        continue

    n_frames = len(entry["energy"])
    n_atoms = Molecule.from_mapped_smiles(smiles, allow_undefined_stereo=True).n_atoms

    for position, gradient, hessian, energy in sorted(
        zip(
            np.asarray(entry["coords"]).reshape(n_frames, n_atoms, 3),
            np.asarray(entry["forces"]).reshape(n_frames, n_atoms, 3),
            np.asarray(entry["hessian"]).reshape(n_frames, n_atoms * 3, n_atoms * 3),
            np.asarray(entry["energy"]),
            strict=True,
        ),
        key=lambda t: t[-1],
    ):

        # Add the data to the graph
        graph_db_add_single_molecule_state(
            graph_db,
            positions=data_to_graph_assignment(
                smiles=smiles,
                data=position.tolist(),
            ),
            gradients=data_to_graph_assignment(
                smiles=smiles,
                data=gradient.tolist(),
            ),
            hessian=hessian.tolist(),
            energy=energy,
        )

        # Create objectives using the data
        objectives.extend(
            [
                # ObjectiveConfigPosition(
                #     GraphDbAddress(
                #         eid=[eid],
                #     ),
                #     scale=100,
                # ),
                # ObjectiveConfigGradient(
                #     GraphDbAddress(
                #         eid=[eid],
                #     ),
                #     scale=1e-5,
                # ),
                ObjectiveConfigHessian(
                    GraphDbAddress(
                        eid=[eid],
                    ),
                    scale=1,
                ),
            ]
        )
        # We'll need this EID for the total energy objective later
        smiles_to_opt_eids.setdefault(smiles, []).append(eid)

        # Increment the entry index into the graph
        eid += 1


# Next, the torsion drives
for entry in list(td_dataset):
    smiles = entry["smiles"]
    if smiles not in smiles_to_add:
        continue

    n_atoms = Molecule.from_mapped_smiles(smiles, allow_undefined_stereo=True).n_atoms
    n_frames = len(entry["energy"])

    eids = list(smiles_to_opt_eids[smiles])
    for coords, energy in zip(
        np.asarray(entry["coords"].reshape(n_frames, n_atoms, 3)),
        np.asarray(entry["energy"]),
        strict=True,
    ):
        positions = data_to_graph_assignment(
            smiles=smiles,
            data=np.asarray(coords),
        )

        graph_db_add_single_molecule_state(
            graph_db,
            positions=positions,
            energy=energy,
        )
        eids.append(eid)
        eid += 1

    # Add all energy objectives for this molecule in one
    # objective to handle QC's arbitrary energy baseline
    energy_total_objective = ObjectiveConfigEnergyTotal(
        GraphDbAddress(
            eid=eids,  # First energy in first eid is reference energy
        ),
        scale=1,
    )
    energy_total_objective.ene_mode = "pairs"
    objectives.append(energy_total_objective)


for obj in objectives:
    obj.verbose = 2

objective_tier = ObjectiveTier()
objective_tier.objectives = dict(enumerate(objectives))

In [9]:
from besmarts.mechanics.fits import gdb_to_physical_systems

physical_systems = gdb_to_physical_systems(graph_db, chemical_system)

2025-04-03 17:24:04.338196 Starting parameterization
2025-04-03 17:24:04.338235 Parameterizing..      1/321Warning, aromaticity set to OEAroModel_MDL with RDKit. Model set to RDKit MDL. Expect differences.
[H:14][C:3]1=[C:2]([N:11]([N:10]=[C:4]1[C:5]23[C:6]([C:7]([C:8]2([H:18])[H:19])([C:9]3([H:20])[H:21])[H:17])([H:15])[H:16])[H:22])[N:1]([H:12])[H:13]
Charges: [H:14][C:3]1=[C:2]([N:11]([N:10]=[C:4]1[C:5]23[C:6]([C:7]([C:8]2([H:18])[H:19])([C:9]3([H:20])[H:21])[H:17])([H:15])[H:16])[H:22])[N:1]([H:12])[H:13] [0.162, -0.3453, 0.0913, -0.0396, -0.5068, 0.3836, -0.1413, -0.0754, -0.1137, -0.0754, 0.0572, 0.0572, -0.0754, 0.0572, 0.0572, 0.0947, 0.0572, 0.0572, 0.3227, -0.7812, 0.3798, 0.3798]
2025-04-03 17:24:04.477353 Parameterizing..      2/321[H:19][c:11]1[c:10]2[c:6]([c:5]([c:4]([c:12]1[C:13]([H:20])([H:21])[Br:14])[C:2](=[O:1])[O:3][H:15])[H:16])[N:7]([N:8]=[C:9]2[H:18])[H:17]
Charges: [H:19][c:11]1[c:10]2[c:6]([c:5]([c:4]([c:12]1[C:13]([H:20])([H:21])[Br:14])[C:2](=[O:1])[O:3][H:15

In [10]:
model = {model.name: i for i, model in enumerate(chemical_system.models)}
model

{'Bonds': 0,
 'Angles': 1,
 'Torsions': 2,
 'OutOfPlanes': 3,
 'Electrostatics': 4,
 'vdW': 5}

In [11]:
split_on_models = {model["Torsions"]: ["t17"]}

In [12]:
from besmarts.mechanics.fits import forcefield_optimization_strategy_default

fitting_strategy = forcefield_optimization_strategy_default(
    chemical_system, models=split_on_models
)
# Configure the strategy to reset new parameters, not to start from the previous ff
# Needs hessians!
fitting_strategy.enable_reset_torsion_stiffness = True
# fitting_strategy.enable_reset_angle_lengths = True
# fitting_strategy.enable_reset_angle_stiffness = True
# fitting_strategy.enable_reset_bond_lengths = True
# fitting_strategy.enable_reset_bond_stiffness = True
# fitting_strategy.enable_reset_outofplane_stiffness = True
fitting_strategy.dihedral_periodicity_reset_alpha = -0.5
fitting_strategy.dihedral_periodicity_reset_max_k = (
    10  # Largest acceptable torsion |k| in ff in kcal/whatever
)
fitting_strategy.dihedral_periodicity_reset_min_k = 0.001
fitting_strategy.dihedral_periodicity_reset_max_n = 5
fitting_strategy.enable_dihedral_periodicity_reset = True

In [13]:
from besmarts.mechanics.fits import chemical_objective

force_field_objective = chemical_objective

```python


reference_ff, reference_objective_value = initial_tier.compute_and_fit(initial_ff)  # P0
for macro_step in strategy.macro_steps: # macro_steps can change dynamically throughout the optimization based on the strategy
    candidate_parameters = macro_step.generate_candidates(reference_ff)  # Micro steps mostly happen in here

    for tier in tiers:
        candidate_parameters = tier.prune_and_fit(candidate_parameters) # Takes best tier.accept

    sorted_candidate_parameters = sort_by( # Best candidates up first
        candidate_parameters,
        initial_tier.compute_and_fit,
    )
    n_params = min(macro_step.parameters_per_macro, len(candidate_parameters))
    candidate_ff, candidate_objective_value = initial_tier.compute_and_fit(
        merge(sorted_candidate_parameters[:n_params]) # if n_params == 1, this is just sorted_candidate_parameters[0]
    ) # If n_params == 1, candidate_objective_value has already been computed in sort_by and that value is reused

    # This macro step has now filtered down to the best force field!
    # But we only pass it on to the next macro step if it's better than the original FF
    if candidate_objective_value < reference_objective_value:
        reference_ff, reference_objective_value = candidate_ff, candidate_objective_value
final_ff, final_objective_value = final_tier.compute_and_fit(candidate_ff)
return (final_ff, final_objective_value)
    

```

In [14]:
from copy import copy

# Fit only the parameters we split on
objective_tier.fit_models = list(split_on_models)
objective_tier.fit_names = None  # Fit all torsions
# Choose the particular parameter values to fit
# omitting this fits all symbols, including eg periodicities and phases
objective_tier.fit_symbols = ["k"]
# Only do 4 rounds of fitting when evaluating candidate parameters
objective_tier.step_limit = 4
objective_tier.minstep = 1e-3
# Suggest parameters with altered periodicities
objective_tier.enable_modify = True
objective_tier.modify_torsion_frequency_limit = fitting_strategy.dihedral_periodicity_reset_max_n
# Can suggest multiple periodicities at the same time by configuring the bounds' bit depth

# Configure the initial objective to not perform a fit, but just to reset the parameters to values based on the hessians
# The full objective is still computed for each force field, but they are not fit
initial_tier = copy(objective_tier)
initial_tier.step_limit = 0  # Number of fitting steps, 0 means don't fit

# Do a proper full fit for the final force field
final_tier = copy(objective_tier)
final_tier.step_limit = 400

In [15]:
from besmarts.core import configs
from besmarts.mechanics.fits import ff_optimize
from contextlib import redirect_stderr, redirect_stdout
import sys
from datetime import datetime

configs.processors = 8
configs.remote_compute_enable = False
configs.workqueue_port = 12345

with open(f"example-ff_optimize-hessians_{datetime.now()}.log", "x") as f:
    with redirect_stderr(f), redirect_stdout(f):
        newcsys, (P0, P), (C0, C) = ff_optimize(
            csys0=chemical_system,
            gdb=graph_db,
            psystems=physical_systems,
            strategy=fitting_strategy,
            chemical_objective=force_field_objective,
            initial_objective=initial_tier,
            tiers=[],
            final_objective=final_tier,
        )